# Counting Rubik's Snake Loops

Loop is a shape where tail and head are adjacent.

Let's define four numbers for $n$-wedge Rubik's Snake:

* L1(n) - number of formulas (i.e. strings of characters 0,1,2,3 of length $n-1$) that describe shape that are loops.
* L2(n) - number of formulas, de-duplicated "up to reversal" that describe loop.
* L3(n) - number of loops (described by n-character formulas) up to cyclic shifts.
* L4(n) - number of loops up to cyclic shifts and reversals.

L4 is the most meaningful number. It is the number of different loops if we look at loops as sets of wedges (ignoring where the tail-head connection is), and consider two loops the same if they can be matched by translation and rotation.

All these values are zeros for odd $n$, and for n=2. So we will be computing values for n=4,6,8,10...

Below are values for n from 4 to 24 that I got (values for odd n omitted, as they are zero).

In [1]:
L1_GOLDEN = [1,8,16,280,2229,20720,226000,2293422,24965960,275633094,3069890660]
L2_GOLDEN = [1,5,10,145,1129,10405,113113,1147142,12484285,137821030,1534958307]
L3_GOLDEN = [1,3,3,31,196,1509,14191,127681,1248963,12531157,127918745]
L4_GOLDEN = [1,2,3,18,112,777,7198,64056,625584,6267820,63970851]

These are all normalized loops (in sense of definition L4) for n=4,6,8,10.

In [2]:
L4_LOOPS = {
  4: {'2222'},
  6: {'131313', '123123'},
  8: {'01230321', '00220022', '12321232'},
  10: {'0131201312', '0021331132', '0021013032', '0013211022', '0231303132', 
       '0121323022', '0230320323', '0113133022', '0012300123', '0113203312', 
       '0123023212', '0131202131', '0013100313', '0021330313', '0120121021', 
       '0013101132', '0022033213', '0231302313'}
}

### Counting the loops

We enumerate all loops using meet-in the middle technique, which is faster than $O(4^n)$.

For L1 we simply count all loops. For L2,L3,L4 we need to "de-duplicate" loops by normalizing them and putting normalized formulas into a set.

In [3]:
import numba
import numpy as np
import time
import functools
from math import gcd

from rubiks_snake import RubiksSnakeCounter, _push_wedge,_pop_wedge,_push_next_wedge_if_can, \
    _prepare_arena, _add_wedges_from_formula_while_can, _pop_n_wedges, _pop_all_but_one, \
    _next_wedge_would_match_head, \
    reverse_encoded_formula, min_cyclic_shift, concat_encoded_formulas, \
    INIT_WEDGE,BOX_SIZE,FACE_IDS_TO_WEDGE_ID,DY


@numba.jit("i8(i8,i8)", inline="always")
def normalize_encoded_loop_L2(f, n):
  f_without_last = f//4
  return min(f_without_last, reverse_encoded_formula(f_without_last, n-1))

@numba.jit("i8(i8,i8)", inline="always")
def normalize_encoded_loop_L3(f, n):
  return min_cyclic_shift(f, n)
        
@numba.jit("i8(i8,i8)", inline="always")
def normalize_encoded_loop_L4(f, n):
  rev = reverse_encoded_formula(f, n)
  return min(min_cyclic_shift(f, n), min_cyclic_shift(rev, n))

@functools.cache
def _count_loops_helper(mode):
  @numba.jit("i8(i8,i8[:],i8[:],i8[:],i8[:])")
  def f(n, formulas_1, last_wedges_1, formulas_2, last_wedges_2):
    half_shapes_count = len(formulas_1)
    assert len(formulas_2) == half_shapes_count
    m = n // 2

    # Manual map to store mapping wedge_id -> List[formula]. 
    linked_list_2 = np.full((half_shapes_count,), -1)
    wedge_id_to_formula_pos_2 = np.full(((BOX_SIZE**3)<<6,), -1)

    for i in range(half_shapes_count):
      last_wedge = last_wedges_2[i]
      if wedge_id_to_formula_pos_2[last_wedge] != -1:
        linked_list_2[i] = wedge_id_to_formula_pos_2[last_wedge] 
      wedge_id_to_formula_pos_2[last_wedge]=i  

    wedges, cubes = _prepare_arena(n, INIT_WEDGE)
    loop_formulas_count = 0 
    normalized_loops = set()
    normalized_loops.add(0)
    normalized_loops.remove(0)

    for i in range(half_shapes_count):
      lookup_key = last_wedges_1[i] ^ 16
      ll_pos = wedge_id_to_formula_pos_2[lookup_key]
      if ll_pos == -1:
        continue
      formula1 = formulas_1[i]
      assert _add_wedges_from_formula_while_can(formula1, m, wedges, cubes) == m
      while ll_pos != -1:
        formula2 = formulas_2[ll_pos]
        ll_pos = linked_list_2[ll_pos]
        formula2_rev = reverse_encoded_formula(formula2, m)      
        added_from_f2 = _add_wedges_from_formula_while_can(formula2_rev, m, wedges, cubes) 
        #assert added_from_f2 <= m-1
        if added_from_f2 == m-1:
          #assert wedges[wedges[0]]>>6 == CENTER_COORD - DY
          if mode == 1:
            loop_formulas_count += 1
          else:
            loop_formula = concat_encoded_formulas(formula1, formula2_rev, m)
            if mode==2:
              norm_formula = normalize_encoded_loop_L2(loop_formula, n)
            elif mode == 3:
              norm_formula = normalize_encoded_loop_L3(loop_formula, n)
            elif mode == 4:
              norm_formula = normalize_encoded_loop_L4(loop_formula, n)   
            normalized_loops.add(norm_formula)
        _pop_n_wedges(added_from_f2, wedges, cubes)    
      _pop_n_wedges(m, wedges, cubes)
      assert wedges[0] == n

    if mode == 1:
      return loop_formulas_count
    else:
      return len(normalized_loops)
  return f
  
# Enumerates all formulas correspdning to loops, no dedup whatsoever.
def count_loops(n, mode=1):
  assert 1<=mode<=4
  if n<4 or n%2==1:
    return 0
  formulas_1, last_wedges_1 = RubiksSnakeCounter.enumerate_shapes(n//2+1, first_wedge_faces=(0,3))
  formulas_2, last_wedges_2 = RubiksSnakeCounter.enumerate_shapes(n//2+1, first_wedge_faces=(3,0))
  return _count_loops_helper(mode)(n, formulas_1, last_wedges_1, formulas_2, last_wedges_2)

In [4]:
for n in range(4,21,2):
  t0=time.time()
  L1,L2,L3,L4 = (count_loops(n, mode=i) for i in range(1,5))
  print("n=%d, L1=%d, L2=%d, L3=%d, L4=%d, time=%fs" % (n,L1,L2,L3,L4, time.time()-t0), flush=True)
  assert L1 == L1_GOLDEN[n//2-2]
  assert L1 == RubiksSnakeCounter.L1[n]
  assert L2 == L2_GOLDEN[n//2-2]
  assert L3 == L3_GOLDEN[n//2-2]
  assert L4 == L4_GOLDEN[n//2-2] 

n=4, L1=1, L2=1, L3=1, L4=1, time=17.520567s
n=6, L1=8, L2=5, L3=3, L4=2, time=0.001617s
n=8, L1=16, L2=10, L3=3, L4=3, time=0.001950s
n=10, L1=280, L2=145, L3=31, L4=18, time=0.002318s
n=12, L1=2229, L2=1129, L3=196, L4=112, time=0.003842s
n=14, L1=20720, L2=10405, L3=1509, L4=777, time=0.014159s
n=16, L1=226000, L2=113113, L3=14191, L4=7198, time=0.128736s
n=18, L1=2293422, L2=1147142, L3=127681, L4=64056, time=1.843187s
n=20, L1=24965960, L2=12484285, L3=1248963, L4=625584, time=27.803878s


In [15]:
t0=time.time()
print(count_loops(18,mode=1), time.time()-t0)

2293422 0.25253844261169434


In [16]:
t0=time.time()
print(count_loops(20,mode=1), time.time()-t0)

24965960 3.1920886039733887


In [17]:
t0=time.time()
print(count_loops(22,mode=1), time.time()-t0)

275633094 47.73346781730652


In [18]:
t0=time.time()
print(count_loops(24,mode=1), time.time()-t0)

3069890660 958.3998811244965


In [19]:
t0=time.time()
print(count_loops(24,mode=4), time.time()-t0)

63970851 1868.8828327655792


### Computing L2 faster using palindromes

Computing L2(n) would take longer than L1(n), and it would require a lot of memory for large n.

However, we can calculate it using the formula $L_2(n)=(L_1(n)+LP(n))/2$, where LP is the number of formulas of length n-1 that are palindromes and describe loops.

In [5]:
t0 = time.time()
for n in range(4,25,2):
  L1 = L1_GOLDEN[n//2-2] 
  L2 = (L1 + RubiksSnakeCounter.count_palindrome_loops(n))//2
  assert L2 == L2_GOLDEN[n//2-2]
  print("L2(%d)=%d" % (n, L2))
print("OK %fs" % (time.time()-t0))

L2(4)=1
L2(6)=5
L2(8)=10
L2(10)=145
L2(12)=1129
L2(14)=10405
L2(16)=113113
L2(18)=1147142
L2(20)=12484285
L2(22)=137821030
L2(24)=1534958307
OK 1.474121s


### Computing L3 faster using Burnside's lemma

Similarly to L2(n), we can compute L3(n) quicker. From Burnside's lemma we get:

$$L_3(n) = \frac{1}{n} \sum_{i=0}^{n-1} X \Big( \text{GCD}(i,n), \frac{n}{\text{GCD}(i,n)} \Big),$$

where $X(n,k)$ is the number of formulas $f$ of length $n$ such that $f^n$ (concatenation of f written k times) describes a loop.

We can compute X as follows:
* $X(n,1) = L_1(n)$ - so we need to pre-compute $L_1(n)$ to compute $L_3(n)$.
* $X(n,k) = 0$ for $k \ge 5$ - this follows from Rubik's Snake geometry.
* $X(n,k)$ for $k=2,3,4$ can be computed by definition by enumerating all formulas of length $n$. Note that to compute $L_1(n)$, the maximum $n$ we need to compute X for is $n/2$, so all these computation will be much faster than computing $L_1(n)$.

In the code below we will also use this convenient definition: *loop-order* of string $f$ is such a number $k$ that $f^k$ describes a loop, or 0 if $f^k$ is not a loop for any $k$ (including the case when f doesn't describe a valid shape).

In [5]:
MAX_POSSIBLE_LOOP_ORDER=4

@numba.jit("i8(i8,i8,i8[:],i8[:])")
def get_loop_order(encoded_formula, formula_len, wedges, cubes):
  ans = 0
  for i in range(MAX_POSSIBLE_LOOP_ORDER):
    added = _add_wedges_from_formula_while_can(encoded_formula, formula_len, wedges, cubes)
    if added < formula_len:
      if added == formula_len-1 and _next_wedge_would_match_head(encoded_formula % 4, wedges):
        ans = i+1
      break
  _pop_all_but_one(wedges, cubes)
  return ans

@functools.cache
@numba.jit("i8[:](i8)")
def compute_X_for_n(n):
  wedges, cubes = _prepare_arena(MAX_POSSIBLE_LOOP_ORDER*n+1, INIT_WEDGE)
  X = np.zeros(MAX_POSSIBLE_LOOP_ORDER+1, dtype=np.int64)
  for f in range(4**n):
    X[get_loop_order(f, n, wedges, cubes)] += 1
  return X  

def X(n, k):
  assert k >= 1
  if k == 1:
    return RubiksSnakeCounter.L1[n]
  elif k>=5:
    return 0
  return compute_X_for_n(n)[k]

t0 = time.time()
for n in range(4,25,2):
  L3 = sum(X(gcd(i,n), n//gcd(i,n)) for i in range(n)) // n
  assert L3 == L3_GOLDEN[n//2-2]
  print("L3(%d)=%d" % (n, L3))
print("OK %fs" % (time.time()-t0))

L3(4)=1
L3(6)=3
L3(8)=3
L3(10)=31
L3(12)=196
L3(14)=1509
L3(16)=14191
L3(18)=127681
L3(20)=1248963
L3(22)=12531157
L3(24)=127918745
OK 2.043338s


### Summary

So, we got these results for the classic 24-wedge snake:

* L1(24) = 3069890660 ~ 3.07 billion
* L2(24) = 1534958307 ~ 1.53 billion
* L3(24) = 127918745 ~ 128.0 million 
* L4(24) = 63970851 ~ 64.0 million

The value of L4 matches the value reported on page 17 in the [Soul of the Snake](https://tinyurl.com/Soul-Of-The-Snake) book. However, I must note that it's incorrect to subtract this value from $6.7 \cdot 10^{12}$ to get number of non-loops, because these are counts of different kinds of objects. Instead, value of L2(24) should be subtracted.